In [ ]:
import sqlalchemy
import os
from bach_open_taxonomy import ObjectivFrame

In [ ]:
dsn = os.environ.get('DSN', 'postgresql://objectiv:@localhost:5432/objectiv')
engine = sqlalchemy.create_engine(dsn, pool_size=1, max_overflow=0)

## set timeframe (date (string or datetime))

In [ ]:
df = ObjectivFrame.from_table(engine, time_aggregation='YYYY-MM-DD', start_date='2021-11-01')
print(df.moment.min().head())
print(df.moment.max().head())

**use normal df**

In [ ]:
# instantiate with or without time_aggregation (if without, aggregation functions from the model hub
#  will aggregated over all data)
df = ObjectivFrame.from_table(engine, time_aggregation='YYYY-MM-DD')

In [ ]:
df.head()

In [ ]:
df.time_aggregation

## unique_users
similar:  unique_sessions

In [ ]:
# using self.time_aggregation
df.model_hub.aggregate.unique_users().head()

In [ ]:
# override time_aggregation
df.mh.agg.unique_sessions(time_aggregation='YYYY-MM').head()

## combine models

In [ ]:
# using same time_aggregations (from self or set)
a = df.mh.agg.unique_sessions() / df.mh.agg.unique_users()
a.head()

In [ ]:
a = df.mh.agg.unique_sessions(time_aggregation='YYYY-MM') / df.mh.agg.unique_users(time_aggregation='YYYY-MM')
a.head()

In [ ]:
# different time_aggregation
# todo: should this work?
# a = df.mh.agg.unique_sessions() / df.mh.agg.unique_users(time_aggregation='YYYY-MM')
# a.head()

## create filter

In [ ]:
a = df.mh.filter.is_first_session()
a.head()

## filtering and combining models

In [ ]:
a = df.mh.agg.unique_users(filter=df.mh.f.is_first_session())
a.head()

## combine with build frame

In [ ]:
# this does not work; as for the filter materialize has to be called
# todo: should this work?
# df.mh.agg.unique_users(filter=df.mh.f.is_first_session()) / df.mh.agg.unique_users()

so we use build_frame to create a frame from series with different base nodes. this is a bit simpler than using merge, as we know how to merge.

In [ ]:
new_users = df.mh.agg.unique_users(filter=df.mh.f.is_first_session()) 
all_users = df.mh.agg.unique_users()
new_df = df.mh.build_frame(new_users, all_users)
new_df.head()

In [ ]:
a = new_df.unique_users_is_first_session / new_df.unique_users
a.head()

## conversion

In [ ]:
df = ObjectivFrame.from_table(engine, time_aggregation='YYYY-MM-DD', start_date='2021-11-16')

In [ ]:
df.conversion_events

**set conversion events**

In [ ]:
df.add_conversion_event(conversion_event='ClickEvent')
df.add_conversion_event(df.location_stack.json[{'_type': 'LinkContext', 'id': 'cta-repo-button'}:].fillna(df.location_stack.json[{'_type': 'LinkContext', 'id': 'GitHub'}:]),
                        conversion_event='ClickEvent')
df.add_conversion_event(df.location_stack.json[{'_type': 'LinkContext', 'id': 'cta-repo-button'}:])

In [ ]:
df.conversion_events

In [ ]:
df.mh.f.conversion('conversion_1').head()

In [ ]:
df[df.mh.f.conversion('conversion_1')].head()

In [ ]:
df[df.mh.f.conversion('conversion_2')].head()

In [ ]:
df[df.mh.f.conversion('conversion_3')].head()

In [ ]:
df.mh.agg.unique_users(filter=df.mh.f.conversion('conversion_2')).head()

In [ ]:
new_df = df.mh.build_frame(df.mh.agg.unique_users(filter=df.mh.f.conversion('conversion_2')), df.mh.agg.unique_users())
new_df['conversion_rate'] = new_df.unique_users_conversion / new_df.unique_users
new_df.head()

## conversion count / conversion funnel

In [ ]:
a = df.mh.f.conversions('conversion_1', 'user_id')

In [ ]:
# different base node, so need to be merged
new_df = df.merge(a, left_index=True, right_index=True)
new_df.head()